In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)



In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')


# File ID of the file you want to attach
file_id='11mrSlmkKS29YAmsO8eynWRArocs1u0uq'

# Path in your Colab environment where you want to save the file
# 11mrSlmkKS29YAmsO8eynWRArocs1u0uq

# Download the file using the file ID
!gdown https://drive.google.com/uc?id={file_id}

# Check if the file is downloaded


Downloading...
From: https://drive.google.com/uc?id=11mrSlmkKS29YAmsO8eynWRArocs1u0uq
To: /content/complaints_processed.csv
100% 99.5M/99.5M [00:02<00:00, 40.3MB/s]


In [ ]:
df = pd.read_csv('/content/complaints_processed.csv')
df.head()

,Unnamed: 0,product,narrative
0,0,credit_card,purchase order day shipping amount receive pro...
1,1,credit_card,forwarded message date tue subject please inve...
2,2,retail_banking,forwarded message cc sent friday pdt subject f...
3,3,credit_reporting,payment history missing credit report speciali...
4,4,credit_reporting,payment history missing credit report made mis...


In [ ]:
df.drop(columns='Unnamed: 0',inplace=True)

In [ ]:
df.head()

,product,narrative
0,credit_card,purchase order day shipping amount receive pro...
1,credit_card,forwarded message date tue subject please inve...
2,retail_banking,forwarded message cc sent friday pdt subject f...
3,credit_reporting,payment history missing credit report speciali...
4,credit_reporting,payment history missing credit report made mis...


In [ ]:
df.isnull().sum()

Unnamed: 0     0
product        0
narrative     10
dtype: int64

In [ ]:
df.dropna(axis=0,inplace=True)

In [ ]:
df['product'].value_counts()

product
credit_reporting       91172
debt_collection        23148
mortgages_and_loans    18990
credit_card            15566
retail_banking         13535
Name: count, dtype: int64

In [ ]:
import re
import string
import nltk

def text_clean(text):
    clean_words = []

    word_list = text.split()
    for word in word_list:
        word_l = word.lower().strip()
        if word_l.isalpha():
            if len(word_l) > 3:
                if word_l not in nltk.corpus.stopwords.words('english'):
                    clean_words.append(word_l)
                else:
                    continue
    return clean_words

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer(analyzer=text_clean)
X=tfidf.fit_transform(df['narrative'])

In [ ]:


from imblearn.under_sampling import RandomUnderSampler

rus = RandomUnderSampler(random_state=42)
X_rus, y_rus = rus.fit_resample(X, df['product'])






In [ ]:
y_rus.shape
print(y_rus.value_counts())

product
credit_card            13535
credit_reporting       13535
debt_collection        13535
mortgages_and_loans    13535
retail_banking         13535
Name: count, dtype: int64


In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X_rus,y_rus,test_size=0.30,random_state=42)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(X_train, y_train)
nb.predict(X_test)

array(['debt_collection', 'credit_card', 'credit_reporting', ...,
       'debt_collection', 'credit_reporting', 'mortgages_and_loans'],
      dtype='<U19')

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,nb.predict(X_test)))

                     precision    recall  f1-score   support

        credit_card       0.76      0.80      0.78      3993
   credit_reporting       0.82      0.78      0.80      4042
    debt_collection       0.85      0.75      0.80      4087
mortgages_and_loans       0.82      0.90      0.86      4072
     retail_banking       0.85      0.87      0.86      4109

           accuracy                           0.82     20303
          macro avg       0.82      0.82      0.82     20303
       weighted avg       0.82      0.82      0.82     20303



In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier()
rf.fit(X_train,y_train)
rf.predict(X_test)

array(['debt_collection', 'credit_card', 'credit_reporting', ...,
       'debt_collection', 'credit_reporting', 'credit_reporting'],
      dtype=object)

In [ ]:
print(classification_report(y_test,rf.predict(X_test)))

                     precision    recall  f1-score   support

        credit_card       0.81      0.80      0.81      3993
   credit_reporting       0.85      0.82      0.84      4042
    debt_collection       0.85      0.83      0.84      4087
mortgages_and_loans       0.87      0.87      0.87      4072
     retail_banking       0.84      0.89      0.86      4109

           accuracy                           0.84     20303
          macro avg       0.84      0.84      0.84     20303
       weighted avg       0.84      0.84      0.84     20303



In [ ]:
from imblearn.over_sampling import SMOTE
x_smote, y_smote = SMOTE().fit_resample(X, df['product'])


In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(x_smote,y_smote,test_size=0.30,random_state=42)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(X_train, y_train)
nb.predict(X_test)

array(['retail_banking', 'credit_reporting', 'retail_banking', ...,
       'retail_banking', 'credit_card', 'retail_banking'], dtype='<U19')

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,nb.predict(X_test)))

                     precision    recall  f1-score   support

        credit_card       0.84      0.86      0.85     27566
   credit_reporting       0.83      0.79      0.81     27439
    debt_collection       0.87      0.79      0.83     27146
mortgages_and_loans       0.87      0.92      0.89     27158
     retail_banking       0.89      0.94      0.91     27449

           accuracy                           0.86    136758
          macro avg       0.86      0.86      0.86    136758
       weighted avg       0.86      0.86      0.86    136758



In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier()
rf.fit(X_train,y_train)
rf.predict(X_test)

array(['credit_reporting', 'credit_reporting', 'retail_banking', ...,
       'retail_banking', 'credit_card', 'retail_banking'], dtype=object)

In [ ]:
print(classification_report(y_test,rf.predict(X_test)))

                     precision    recall  f1-score   support

        credit_card       0.96      0.97      0.96     27566
   credit_reporting       0.95      0.93      0.94     27439
    debt_collection       0.95      0.96      0.95     27146
mortgages_and_loans       0.97      0.96      0.97     27158
     retail_banking       0.98      0.99      0.99     27449

           accuracy                           0.96    136758
          macro avg       0.96      0.96      0.96    136758
       weighted avg       0.96      0.96      0.96    136758

